# Batch Scoring on IBM Cloud Pak for Data (ICP4D)

We are going to use this notebook to create and/or run a batch scoring job against a model that has previously been created and deployed to the Watson Machine Learning (WML) instance on Cloud Pak for Data (CP4D).

## 1.0 Install required packages


There are a couple of Python packages we will use in this notebook. We will install/upgrade the Watson Machine Learning (WML) client library to interact with the Watson Machine Learning service from this notebook. These package may already be installed by default on CP4D.

- WML Client: http://ibm-wml-api-pyclient.mybluemix.net/

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install --user ibm-watson-machine-learning==1.0.53 --upgrade | tail -n 1

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

## 2.0 Create Batch Deployment Job

### 2.1 Instantiate Watson Machine Learning Client

To interact with the local Watson Machine Learning instance, we will be using the Python SDK. 

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

<font color='red'>Replace the `username` and `password` values of `************` with your Cloud Pak for Data `username` and `password`. The value for `url` should match the `url` for your Cloud Pak for Data cluster, which you can get from the browser address bar (be sure to include the 'https://'.</font> The credentials should look something like this (these are example values, not the ones you will use):

`
wml_credentials = {
    "url": "https://zen.clusterid.us-south.containers.appdomain.cloud/",
    "username": "cp4duser",
    "password" : "cp4dpass",
    "instance_id": "wml_local",
    "version" : "3.5"
}
`
#### NOTE: Make sure that there is no trailing forward slash `/` in the `url`

In [ ]:
# Be sure to update these credentials before running the cell.
wml_credentials = {
                   "url": "************",
                   "username": "************",
                   "password" : "************",
                   "instance_id": "wml_local",
                   "version" : "3.5"
                  }

wml_client = APIClient(wml_credentials)

In [ ]:
wml_client.spaces.list()

### 2.2 Find Deployment Space

We will try to find the `ID` for the deployment space you want to use and set it as the default space for the client.

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

- Update with the name of the deployment space where you have created the batch deployment.

In [ ]:
# Be sure to update the name of the space with the one you want to use.
DEPLOYMENT_SPACE_NAME = '************'

In [ ]:
all_spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in all_spaces:
    if space['entity']['name'] == DEPLOYMENT_SPACE_NAME:
        space_id = space["metadata"]["id"]
        print("\nDeployment Space ID: ", space_id)

if space_id is None:
    print("WARNING: Your space does not exist. Create a deployment space before proceeding.")
    # We could programmatically create the space.
    #space_id = wml_client.spaces.store(meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]

In [ ]:
# Now set the default space to the ID for your deployment space. If this is successful, you will see a 'SUCCESS' message.
wml_client.set.default_space(space_id)

In [ ]:
# These are the models and deployments we currently have in our deployment space.
wml_client.repository.list_models()
wml_client.deployments.list()

### 2.3 Find Batch Deployment

We will try to find the batch deployment which was created.

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

- Update the variable with the name of the batch deployment you created previously.

In [ ]:
DEPLOYMENT_NAME = '************'

In [ ]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
deployment_details = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['id']
        deployment_details = deployment
        #print(json.dumps(deployment_details, indent=3))
        break

print("Deployment id: {}".format(deployment_uid))
wml_client.deployments.get_details(deployment_uid)

### 2.4 Get Batch Test Data

We will load some data to run the batch predictions.

In [ ]:
import pandas as pd

from project_lib import Project
project = Project.access()

batch_set = pd.read_csv(project.get_file('German-Credit-Risk-SmallBatchSet.csv'))
batch_set = batch_set.drop('CustomerID', axis=1,errors = 'ignore')
batch_set.head()

### 2.5 Create Job

We can now use the information about the deployment and the test data to create a new job against our batch deployment. We submit the data as inline payload and want the results (i.e predictions) stored in a CSV file.

In [ ]:
import time
timestr = time.strftime("%Y%m%d_%H%M%S")
job_payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': batch_set.columns.values.tolist(),
        'values': batch_set.values.tolist()
    }],
    wml_client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
            "type": "data_asset",
            "connection": {},
            "location": {
                "name": "batchres_{}_{}.csv".format(timestr,deployment_uid),
                "description": "results"
            }
    }
}

job = wml_client.deployments.create_job(deployment_id=deployment_uid, meta_props=job_payload)
job_uid = wml_client.deployments.get_job_uid(job)

print('Job uid = {}'.format(job_uid))

In [ ]:
wml_client.deployments.list_jobs()

## 3.0 Monitor Batch Job Status

The batch job is an async operation. We can use the identifier to track its progress. Below we will just poll until the job completes (or fails).

In [ ]:
def poll_async_job(client, job_uid):
    import time
    while True:
        job_status = client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status['state']
        if state == 'completed' or 'fail' in state:
            return client.deployments.get_job_details(job_uid)
        time.sleep(5)
            
job_details = poll_async_job(wml_client, job_uid)

In [ ]:
wml_client.deployments.list_jobs()

### 3.1 Check Results

With the job complete, we can see the predictions. 

In [ ]:
wml_client.deployments.get_job_details()

In [ ]:
print(json.dumps(job_details, indent=2))

## Congratulations, you have created and submitted a job for batch scoring !